In [ ]:
from typing import List

# =========================================
# CONFIGURAÇÕES
# =========================================

CATALOG = ""  # schemas aviacao_* estão no metastore padrão, sem catálogo

SCHEMAS_TO_CLEAN: List[str] = [
    "aviacao_landing",
    "aviacao_bronze",
    "aviacao_silver",  # <--- NOVO: limpa a Silver também
    "aviacao_meta",
]

META_SCHEMA = "aviacao_meta"

# Tabelas de meta/watermark que queremos garantir que sejam dropadas
WM_TABLE_NAMES: List[str] = [
    "watermark_incremental",     # legado (se ainda existir)
    "silver_bronze_watermark",   # watermark Silver x Bronze (nova)
    "bronze_landing_batches",    # meta da Bronze (batches processados)
]

# Config do Volume UC usado pela landing
VOLUME_CATALOG = "main"
VOLUME_SCHEMA = "engenharia_dados"
VOLUME_NAME = "aviacao_landing"

LANDING_CSV_BASE_PATH = f"/Volumes/{VOLUME_CATALOG}/{VOLUME_SCHEMA}/{VOLUME_NAME}/aviacao/landing"


# =========================================
# HELPERS
# =========================================

def qschema(schema: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}"
    return schema


def qtable(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"


def qname(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"


# =========================================
# 1) RESETAR CARGA INCREMENTAL (WATERMARKS/META)
# =========================================

print("\n>>> Drop tabelas de watermark/meta (reset incremental):")
for wm_name in WM_TABLE_NAMES:
    wm_full_name = qname(META_SCHEMA, wm_name)
    print(f" - DROP TABLE IF EXISTS {wm_full_name}")
    try:
        spark.sql(f"DROP TABLE IF EXISTS {wm_full_name}")
        print(f"   OK: {wm_full_name} removida (se existia).")
    except Exception as e:
        print(f"   Erro ao dropar {wm_full_name}: {e}")


# =========================================
# 2) DROPAR TODAS AS TABELAS DOS SCHEMAS
# =========================================

for schema in SCHEMAS_TO_CLEAN:
    schema_full = qschema(schema)
    print(f"\n=== Limpando tabelas do schema: {schema_full} ===")

    try:
        tables_df = spark.sql(f"SHOW TABLES IN {schema_full}")
    except Exception as e:
        print(f"Schema {schema_full} não existe ou sem permissão. Detalhe: {e}")
        continue

    for row in tables_df.collect():
        tbl_name = row["tableName"]
        fq_name = qtable(schema, tbl_name)
        print(f"DROP TABLE {fq_name}")
        spark.sql(f"DROP TABLE IF EXISTS {fq_name}")


# =========================================
# 3) OPCIONAL: DROPAR OS SCHEMAS
# =========================================
# Se quiser manter os schemas vazios, comente o bloco abaixo.

for schema in SCHEMAS_TO_CLEAN:
    schema_full = qschema(schema)
    print(f"\nDROP SCHEMA {schema_full} CASCADE")
    try:
        spark.sql(f"DROP SCHEMA IF EXISTS {schema_full} CASCADE")
    except Exception as e:
        print(f"Erro ao dropar schema {schema_full}: {e}")


# =========================================
# 4) APAGAR ARQUIVOS CSV DA LANDING
# =========================================

print(f"\nRemovendo arquivos da landing em: {LANDING_CSV_BASE_PATH}")
try:
    dbutils.fs.rm(LANDING_CSV_BASE_PATH, True)
    print("Arquivos CSV da landing removidos.")
except Exception as e:
    print(f"Não foi possível remover {LANDING_CSV_BASE_PATH}: {e}")

print("\n=== Ambiente de aviação zerado (landing, bronze, silver, meta e incremental) ===")
